# Replicability and transparency in topic modelling: developing best practice guidelines for the digital humanities

_Copyright (c) 2023 [Andressa Gomide, Mathew Gillings, Diego Gimenez]_

This file is part of Gomide et al. 2023.

This project is licensed under the terms of the MIT license.

@get_gutemberg
This script download books from the Gutemberg Project (https://www.gutenberg.org/), 
removes unecessary elements (e.g. boilerplates, page numbers),
extract the metadata for each book
and saves:
- the original book file (html)
- the cleaned content (txt)
- the metadata (tsv)


Functions in this file
    * download_url - it takes an argument: 
    a string with with url path
    and returns the content of the url as bytes

## Import libraries
if it doesn't require a lot of work, it might be a better idea to use your own code, as you have more awareness and avoid having a lot of dependencies. In this example, its better to use our own codes than importing https://pypi.org/project/Gutenberg/

In [1]:
import re # for regular expressions
from urllib.request import urlopen # to request the content from the internet
from bs4 import BeautifulSoup # to work with html files (bs4 is known to be user friendly)
import pandas as pd # to store metadata as dataframe

In [2]:
def download_url(urlpath):
    ''' 
    download content from an url address
    Args: 
        urlpath (str): the url path
    Returns:
        connection.read() (bytes): the content of the page 
    '''
    try:
        # open a connection to the server
        with urlopen(urlpath, timeout=3) as connection:
            # return content of the url read as bytes
            return connection.read()
    except:
        # return None
        print(f"There was an issue when trying to download{urlpath}")

create a list with the books id.
the id can be retrieved at https://www.gutenberg.org/

In [3]:
book_id_list = ["44540", "55682", "31971"]

- sometimes the same data content is available in different formats. it is a good idea to test extracting two different formats to get an idea which one will be better for the project.
- it is almost always easier to work with plain text, but preserving section breaks can lead to further analysis
- in our case, getting the data from html format sounds better and easier to (a) preserve the sections boundaries (b) to make cleaning easier

# getting the books from the plain format

we first create a data frame that will serve to store the metadata

In [4]:
df = pd.DataFrame(columns = ['author', 'title', 'lang', 'subj', 'datepub'])

then we go through the list of book ids

In [5]:
for book_id in book_id_list:
    # url for plain text book
    url_plain = f'https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt'

    # download the content
    data_plain = download_url(url_plain)

    # plain text link doesnt include metadata. 
    # we have to go to the previous page
    url_meta = f'https://www.gutenberg.org/ebooks/{book_id}'
    metadata = download_url(url_meta)

    # parse document 
    soup = BeautifulSoup(metadata, 'html.parser')

    # get metadata
    author = soup.find('a', {'about': re.compile(r'\/authors\/.*')}).text
    lang = soup.find('a', {'href': re.compile(r'\/browse\/languages\/.*')}).text
    subj = soup.find('a', {'href': re.compile(r'\/ebooks\/subject\/*')}).text
    title = soup.find('td', {'itemprop': 'headline'}).text
    datepub = soup.find('td', {'itemprop': 'datePublished'}).text

    # remove line breaks
    meta_list = [sub.replace('\n', '') for sub in [author, title, lang, subj, datepub]]


    # df.loc[book_id] = [book_id, meta_list[0], meta_list[1], meta_list[2], meta_list[3], meta_list[4]]
    df.loc[book_id] = [meta_list[0], meta_list[1], meta_list[2], meta_list[3], meta_list[4]]

    # write book content to file
    with open(f"input/{book_id}.txt", 'wb') as file:
        file.write(data_plain)

and save the metadata as a tsv file

In [6]:
# see the data
print(df)

# write metadata to file
df.to_csv('output/books_metadata.tsv', sep='\t', encoding='utf-8')

                                       author  \
44540  Alencar, José Martiniano de, 1829-1877   
55682             Machado de Assis, 1839-1908   
31971              Queirós, Eça de, 1845-1900   

                                               title        lang  \
44540                                  Cinco minutos  Portuguese   
55682                                  Quincas Borba  Portuguese   
31971  O crime do padre Amaro, scenas da vida devota  Portuguese   

                                                    subj       datepub  
44540                                            Fiction  Dec 29, 2013  
55682  Brazil -- History -- Empire, 1822-1889 -- Fiction   Oct 5, 2017  
31971                                Portugal -- Fiction  Apr 13, 2010  


## to get the books from html
# create empty df to store the metadata

In [7]:
df = pd.DataFrame(columns = ['author', 'title', 'lang', 'subj', 'datepub'])

for book_id in book_id_list:
    url_html = f'https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}-images.html'
    data_html = download_url(url_html)

    # parse
    soup = BeautifulSoup(data_html, 'html.parser')

    # get metadata
    author = soup.find('meta', {'name' : 'AUTHOR'})['content'] if soup.find('meta', {'name' : 'AUTHOR'}) is not None else 'NA'
    lang = soup.find('meta', {'name' : 'dc.language'})['content'] if soup.find('meta', {'name' : 'dc.language'}) is not None else 'NA'
    subj = soup.find('meta', {'name' : 'dc.subject'})['content'] if soup.find('meta', {'name' : 'dc.subject'}) is not None else 'NA'
    title = soup.find('meta', {'property' : 'og:title'})['content'] if soup.find('meta', {'property' : 'og:title'}) is not None else 'NA'
    datepub = soup.find('meta', {'name' : 'dcterms.created'})['content'] if soup.find('meta', {'name' : 'dcterms.created'}) is not None else 'NA'

    ## remove unnecessary elements
    # style
    for i in soup.find_all('style'):
        i.decompose()

    # boiler plates
    for i in soup.find_all('section', {'class': re.compile('.*boilerplate.*')}):
        i.decompose()

    # editor comments
    for i in soup.find_all('div', {'class': 'fbox'}):
        i.decompose()

    # page numbers
    for i in soup.find_all('span', {'class': 'pagenum'}):
        i.decompose()

    # remove br tags
    for i in soup.find_all('br'):
        i.unwrap()

    # remove head
    soup.find('head').decompose()

    # get metadata
    df.loc[book_id] = [author, title, lang, subj, datepub]


    # write to file with tags
    with open(f'input/html/{book_id}.html', 'w', encoding = 'utf-8') as file:
        file.write(str(soup.prettify()))
    # write to file without tags
    with open(f'input/plain/{book_id}.txt', 'w', encoding = 'utf-8') as file:
        file.write(soup.text)

print(df)
# write metadata to file
df.to_csv('output/books_metadata.tsv', sep='\t', encoding='utf-8')

                           author  \
44540  José Martiniano de Alencar   
55682                          NA   
31971              Eça de Queirós   

                                               title lang  \
44540                                  Cinco minutos   pt   
55682                                  Quincas Borba   pt   
31971  O crime do padre Amaro, scenas da vida devota   pt   

                                                    subj     datepub  
44540                                            Fiction  2013-12-29  
55682  Brazil -- History -- Empire, 1822-1889 -- Fiction  2017-10-05  
31971                                Portugal -- Fiction  2010-04-13  
